# setup

In [3]:
import sys
sys.path.append('../../..')

In [4]:
from copulae.input import generate_copula_net_input
from copulae.training.cflax.mixtures import DoubleIntegral
from copulae.training.cflax.mixtures import MixtureCopula
from copulae.training.cflax.mixtures import FGMCopNet

from copulae.training.loss import sq_error, sq_error_partial, copula_likelihood
from copulae.training import setup_training

In [5]:
def build_mixture(base_class, n, *args, **kwargs):
    rv = []
    for i in range(n):
        rv.append(base_class(*args, **kwargs))
    return MixtureCopula(rv)

In [6]:
import jax.numpy as jnp
import jax.scipy.stats as jss
import jax

from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import optax
import pickle
from scipy.stats import bootstrap

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# data
https://github.com/yutingng/gen-AC.git

In [8]:
import pandas as pd
import scipy
import copy

from sklearn.model_selection import train_test_split

In [9]:
def add_train_random_noise(data, num_adds):
  new_data = np.random.rand(num_adds, data.shape[1])
  return np.concatenate((data, new_data), axis = 0)

In [10]:
def rank_normalization(X):
  X = copy.deepcopy(X)
  for z in X:
      ndata = z.shape[0]
      gap = 1./(ndata+1)
      nfeats = z.shape[1]
      for i in range(nfeats):
          z[:, i] = scipy.stats.rankdata(z[:, i], 'ordinal')*gap
  return X

In [11]:
!git clone https://github.com/yutingng/gen-AC.git

Cloning into 'gen-AC'...
remote: Enumerating objects: 466, done.
remote: Counting objects: 100% (466/466), done.
remote: Compressing objects: 100% (339/339), done.
remote: Total 466 (delta 159), reused 421 (delta 123), pack-reused 0
Receiving objects: 100% (466/466), 10.28 MiB | 33.52 MiB/s, done.
Resolving deltas: 100% (159/159), done.


In [12]:
class Boston():
  def __init__(self):
    # read
    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep = "\s+", skiprows = 22, header = None)
    X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    y = raw_df.values[1::2, 2]

    # split
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, random_state = 142857)
    X_train = np.concatenate((X_train, y_train[:, None]), axis = 1)
    X_test  = np.concatenate((X_test, y_test[:, None]), axis = 1)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # noise
    X_train = add_train_random_noise(X_train, int(X_train.shape[0]*0.01))

    # 2d
    train_data = X_train[:, [0, 13]]
    test_data = X_test[:, [0, 13]]

    # flip
    train_data[:, 0] = 1 - train_data[:, 0]
    test_data[:, 0] = 1 - test_data[:, 0]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [13]:
class INTC_MSFT():
  def __init__(self):
    # read
    intel_f = open('gen-AC/data/raw/INTC_MSFT_GE/INTEL.data', 'r')
    intel = np.array(list(map(float, intel_f.readlines())))

    ms_f = open('gen-AC/data/raw/INTC_MSFT_GE/MS.data', 'r')
    ms = np.array(list(map(float, ms_f.readlines())))

    ge_f = open('gen-AC/data/raw/INTC_MSFT_GE/GE.data', 'r')
    ge = np.array(list(map(float, ge_f.readlines())))

    # split
    X = np.concatenate((intel[:, None], ms[:, None]), axis = 1)
    X_train, X_test, _, _ = train_test_split(X, X, shuffle = True, random_state = 142857)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # 2d, noise
    train_data = X_train[:, [0, 1]]
    train_data = add_train_random_noise(train_data, int(train_data.shape[0]*0.01))
    test_data = X_test[:, [0, 1]]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [14]:
class GOOG_FB():
  def __init__(self):
    # read
    goog_f = open('gen-AC/data/raw/FB_GOOG/goog/close.vals', 'r')
    goog = np.array(list(map(float, goog_f.readlines())))

    fb_f = open('gen-AC/data/raw/FB_GOOG/fb/close.vals', 'r')
    fb = np.array(list(map(float, fb_f.readlines())))

    # split
    X = np.concatenate((goog[:, None], fb[:, None]), axis = 1)
    X_train, X_test, _, _ = train_test_split(X, X, shuffle=True, random_state=142857)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # 2d, noise
    train_data = X_train[:, [0, 1]]
    train_data = add_train_random_noise(train_data, int(train_data.shape[0]*0.01))
    test_data = X_test[:, [0, 1]]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [15]:
def get_set(D_val, data_points):
  points = D_val
  points = jnp.expand_dims(points, axis=0)

  # PDF and CDF for X
  kde_x = jss.gaussian_kde(data_points[0], bw_method='silverman')
  density_x = kde_x.evaluate(points[0, 0, :])
  cumulative_x = jnp.array([kde_x.integrate_box_1d(-jnp.inf, p) for p in points[0, 0, :]])

  # PDF and CDF for Y
  kde_y = jss.gaussian_kde(D[1], bw_method='silverman')
  density_y = kde_y.evaluate(points[0, 1, :])
  cumulative_y = jnp.array([kde_y.integrate_box_1d(-jnp.inf, p) for p in points[0, 1, :]])

  I_pdf = density_x.T * density_y.T
  I_pdf = jnp.expand_dims(I_pdf, axis=0)
  cdf_xy = jnp.array((cumulative_x, cumulative_y))
  cdf_xy = jnp.expand_dims(cdf_xy, axis=0)

  del density_x
  del density_y
  del cumulative_x
  del cumulative_y

  return points, I_pdf, cdf_xy

In [16]:
np.random.seed(30091985)
key = jax.random.PRNGKey(30091985)

# params

In [17]:
layer_widths = [128, 64, 32, 16]
losses = [
    (0.01, sq_error),
    (0.5, sq_error_partial),
    (0.1, copula_likelihood),
]
lr = 2e-3
n_iter = 10000
label = '2catspfgm10'

In [18]:
def L_d(losses, params, state):
  loss = jnp.zeros((1,len(losses)), dtype=jnp.float32)
  for i, (w, loss_func) in enumerate(losses):
    loss = loss.at[0, i].set(w * loss_func(params, state))
  return loss

losses_eval = [
    (1.0, sq_error),
    (1.0, sq_error_partial),
    (1.0, copula_likelihood),
]

# boston

In [19]:
model = DoubleIntegral(
    build_mixture(FGMCopNet, 10)
)

In [20]:
data_loader = Boston()
D = np.array([data_loader.train_x, data_loader.train_y])[:, :, 0]
TrainingTensors = generate_copula_net_input(
    D=D,
    bootstrap=False
)

In [21]:
nn_C, nn_dC, nn_c, cop_state, forward, grad = setup_training(
    model, TrainingTensors, losses
)

In [22]:
key, subkey = jax.random.split(key)
init_params = model.init(subkey, TrainingTensors.UV_batches[0])
del subkey

params = init_params
optimizer = optax.adam(lr)
opt_state = optimizer.init(params)

In [23]:
best = 1e6
for i in tqdm(range(n_iter)):
    grads, cop_state = grad(params, cop_state)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        loss = L_d(losses_eval, params, cop_state)
        if loss[0][-1] < best:
          best_params = params
          best_cop_state = cop_state
          best = loss[0][-1]
        print('Iter {}. Loss {}'.format(i, loss))

  0%|          | 1/10000 [00:19<53:57:56, 19.43s/it]

Iter 0. Loss [[4.78042    0.48419365 2.9546013 ]]


  0%|          | 14/10000 [00:36<2:37:13,  1.06it/s]

Iter 10. Loss [[4.7995005 0.4897014 2.843069 ]]


  0%|          | 23/10000 [00:36<52:05,  3.19it/s]  

...

100%|█████████▉| 9984/10000 [10:49<00:00, 17.76it/s]

Iter 9980. Loss [[4.895995   0.52294165 2.1954    ]]


100%|█████████▉| 9993/10000 [10:50<00:00, 18.88it/s]

Iter 9990. Loss [[4.895995   0.52294165 2.1954    ]]


100%|██████████| 10000/10000 [10:51<00:00, 15.36it/s]


In [24]:
density_graph_points, I_pdf, cdf_xy = get_set(D, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))
np.mean(yhat)

1.6159425

In [25]:
D_val = np.array([data_loader.validation_x, data_loader.validation_y])[:, :, 0]
density_graph_points, I_pdf, cdf_xy = get_set(D_val, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))

L_d(losses_eval, best_params, best_cop_state), np.mean(yhat)

(Array([[4.895995 , 0.5229416, 2.1954   ]], dtype=float32), 2.0659723)

In [26]:
res = bootstrap(yhat, np.mean)
res.standard_error, res.confidence_interval

(0.48603508,
 ConfidenceInterval(low=1.2237554876559325, high=3.1573607667089054))

In [27]:
with open('drive/MyDrive/copulae/models/{}_boston.pkl'.format(label), 'wb') as fp:
    pickle.dump(best_params, fp)

# intc-msft

In [28]:
model = DoubleIntegral(
    build_mixture(FGMCopNet, 10)
)

In [29]:
data_loader = INTC_MSFT()
D = np.array([data_loader.train_x, data_loader.train_y])[:, :, 0]
TrainingTensors = generate_copula_net_input(
    D=D,
    bootstrap=False
)

In [30]:
nn_C, nn_dC, nn_c, cop_state, forward, grad = setup_training(
    model, TrainingTensors, losses
)

In [31]:
key, subkey = jax.random.split(key)
init_params = model.init(subkey, TrainingTensors.UV_batches[0])
del subkey

params = init_params
optimizer = optax.adam(lr)
opt_state = optimizer.init(params)

In [32]:
best = 1e6
for i in tqdm(range(n_iter)):
    grads, cop_state = grad(params, cop_state)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        loss = L_d(losses_eval, params, cop_state)
        if loss[0][-1] < best:
          best_params = params
          best_cop_state = cop_state
          best = loss[0][-1]
        print('Iter {}. Loss {}'.format(i, loss))

  0%|          | 1/10000 [00:20<57:03:22, 20.54s/it]

Iter 0. Loss [[ 5.6808386  1.9940269 -1.7908527]]


  0%|          | 12/10000 [00:37<2:34:30,  1.08it/s]

Iter 10. Loss [[ 5.658046   1.9496424 -1.7808523]]


  0%|          | 22/10000 [00:38<40:17,  4.13it/s]

...

100%|█████████▉| 9983/10000 [11:58<00:01, 16.09it/s]

Iter 9980. Loss [[ 4.7287674  0.6311902 -0.8746538]]


100%|█████████▉| 9993/10000 [11:58<00:00, 17.24it/s]

Iter 9990. Loss [[ 4.7287674   0.6311902  -0.87465394]]


100%|██████████| 10000/10000 [11:59<00:00, 13.90it/s]


In [33]:
density_graph_points, I_pdf, cdf_xy = get_set(D, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))
np.mean(yhat)

-1.0918711

In [34]:
D_val = np.array([data_loader.validation_x, data_loader.validation_y])[:, :, 0]
density_graph_points, I_pdf, cdf_xy = get_set(D_val, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))

L_d(losses_eval, best_params, best_cop_state), np.mean(yhat)

(Array([[ 5.6808386,  1.9940269, -1.7908527]], dtype=float32), -1.0747359)

In [35]:
res = bootstrap(yhat, np.mean)
res.standard_error, res.confidence_interval

(0.024255976,
 ConfidenceInterval(low=-1.1212191367159137, high=-1.0262523484423476))

In [36]:
with open('drive/MyDrive/copulae/models/{}_intcmsft.pkl'.format(label), 'wb') as fp:
    pickle.dump(best_params, fp)

# goog-fb

In [37]:
model = DoubleIntegral(
    build_mixture(FGMCopNet, 10)
)

In [38]:
data_loader = GOOG_FB()
D = np.array([data_loader.train_x, data_loader.train_y])[:, :, 0]
TrainingTensors = generate_copula_net_input(
    D=D,
    bootstrap=False
)

In [39]:
nn_C, nn_dC, nn_c, cop_state, forward, grad = setup_training(
    model, TrainingTensors, losses
)

In [40]:
key, subkey = jax.random.split(key)
init_params = model.init(subkey, TrainingTensors.UV_batches[0])
del subkey

params = init_params
optimizer = optax.adam(lr)
opt_state = optimizer.init(params)

In [41]:
best = 1e6
for i in tqdm(range(n_iter)):
    grads, cop_state = grad(params, cop_state)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        loss = L_d(losses_eval, params, cop_state)
        if loss[0][-1] < best:
          best_params = params
          best_cop_state = cop_state
          best = loss[0][-1]
        print('Iter {}. Loss {}'.format(i, loss))

  0%|          | 1/10000 [00:20<56:54:03, 20.49s/it]

Iter 0. Loss [[ 4.0999684  1.1226375 -1.3828143]]


  0%|          | 13/10000 [00:38<2:33:13,  1.09it/s]

Iter 10. Loss [[ 4.080251   1.0986565 -1.364092 ]]


  0%|          | 23/10000 [00:38<31:27,  5.28it/s]

...

100%|█████████▉| 9983/10000 [12:05<00:01, 15.68it/s]

Iter 9980. Loss [[ 3.710568    0.7125539  -0.75415087]]


100%|█████████▉| 9993/10000 [12:06<00:00, 17.21it/s]

Iter 9990. Loss [[ 3.7105672   0.71255356 -0.7541492 ]]


100%|██████████| 10000/10000 [12:06<00:00, 13.76it/s]


In [42]:
density_graph_points, I_pdf, cdf_xy = get_set(D, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))
np.mean(yhat)

-0.8119479

In [43]:
D_val = np.array([data_loader.validation_x, data_loader.validation_y])[:, :, 0]
density_graph_points, I_pdf, cdf_xy = get_set(D_val, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))

L_d(losses_eval, best_params, best_cop_state), np.mean(yhat)

(Array([[ 4.0999684,  1.1226375, -1.3828143]], dtype=float32), -0.8132408)

In [44]:
res = bootstrap(yhat, np.mean)
res.standard_error, res.confidence_interval

(0.015619219,
 ConfidenceInterval(low=-0.8407007004917256, high=-0.7793216440958611))

In [45]:
with open('drive/MyDrive/copulae/models/{}_googfb.pkl'.format(label), 'wb') as fp:
    pickle.dump(best_params, fp)